# Digital Signal Processing
## Convolution and Correlation

### MsCV - ViBOT - MAIA

In [4]:
%matplotlib inline
%pprint off

# To make some nice plot
import matplotlib.pyplot as plt
# To play with arrays
import numpy as np
from skimage import io
from skimage import img_as_float
from matplotlib import cm
from array import array

Pretty printing has been turned ON


## Exercise 1- 1D Convolution

**1.1** Defining a signal $x$ and an impulse response $h$ as parameters, write a program `conv_fn` to compute the convolution product $y=h*x$. Do not use the python or matlab function and just consider zero padding to solve the border problem.

**1.2** Test your function, while using the following signal ($x[n] = \{\underline{1},2,3,4\}$) and following filters, ($\delta[k]$, $H[k]$, $e^{k}$, and $h = \{-1, 1\}$), respectively.

**1.3** Consider the previous input signal, write it's extended version while considering `symmetry`, `periodic` and `constant values` padding and $h = H[k-5]$



In [168]:
def conv_fn(x,h):
    #f = np.append([0,0,0,0,1,x,0,0,0,0])
    r = np.zeros(len(x)+2*len(h))
    f = np.zeros(len(x)+2*len(h))
    g = np.zeros(len(x)+2*len(h))
    g[0:len(h)] = h.copy()
    f[len(h):len(h)+len(x)] = x.copy()
    #print g
    #print f
    for k in range(len(g)):
        c = []
        for p in range(len(g)):
            r[k] = r[k] + (f[p] * g[k-p])
            c.append(g[k-p])
            
            
        print c 
        print f
    return r


#Impulse
def dirac(n=0, N=20):
    if n> N-1:
        raise ValueError('n should be inferior to N-1')
        
        
    s = np.zeros((N,))
    s[n] = 1
    
    return s

#H(k)

def unit(n=0, N=20):
    if n> N-1:
        raise ValueError('n should be inferior to N-1')
        
        
    u = np.zeros((N,))
    
    u[n : N] =1
    
    return u


In [169]:

x = np.array([1,2,3,4])
#conv_fn(x,dirac(5))
print unit(0,4)
y = conv_fn(x,unit(0,4))
print y
y = np.convolve(x,unit(0,4))
print y

[ 1.  1.  1.  1.]
[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0]
[ 0.  0.  0.  0.  1.  2.  3.  4.  0.  0.  0.  0.]
[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0]
[ 0.  0.  0.  0.  1.  2.  3.  4.  0.  0.  0.  0.]
[1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]
[ 0.  0.  0.  0.  1.  2.  3.  4.  0.  0.  0.  0.]
[1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 0.  0.  0.  0.  1.  2.  3.  4.  0.  0.  0.  0.]
[0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 0.  0.  0.  0.  1.  2.  3.  4.  0.  0.  0.  0.]
[0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 0.  0.  0.  0.  1.  2.  3.  4.  0.  0.  0.  0.]
[0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[ 0.  0.  0.  0.  1.  2.  3.  4.  0.  0.  0.  0.]
[0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[ 0.  0.  0.  0.  1.  2.  3.  4.  0.  0.  0.  0.]
[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0]
[ 0.  0.  0.  0.  1.  2.  3.  4. 

## Exercise 2- 2D Convolution

**2.1** Define a 2D convolution function (Only consider zero-padding the image)

**2.2** Using your function, smooth the `lena` image with the following Gaussian kernel:
$$ K = \frac{1.}{256}\begin{bmatrix} 1 & 4 & 6 & 4 & 1 \\ 4 & 16 & 24 & 16 & 4 \\ 6 & 24 & 36 & 24 & 6 \\ 4 & 16 & 24 & 16 & 4 \\ 1 & 4 & 6 & 4 & 1 \end{bmatrix}$$

**2.3** Apply sobel filtering using convolution


## Exercise 3 - Character recognition using cross-correlation

**3.1** load `a.png` and `text.png` images and binarized them using [`threshold_otsu`](http://scikit-image.org/docs/dev/api/skimage.filters.html?highlight=filters#skimage.filters.threshold_otsu)

**3.2** Use [`signal.correlate2d`](https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.signal.correlate2d.html#scipy.signal.correlate2d) from `scipy` to correlate your binarized images.
**Note, signal correspond to the bright pixels within and image (invert your masks)**

**3.3** Find the first 10 characters recognized as `a`.